In [14]:
!pip install openai tiktoken chromadb langchain-community gitpython

In [15]:
#Se importan los módulos
from langchain.schema import Document
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings


In [16]:
#Diccionario de productos
productos={
   "Model X": {"especificacions": "Pantalla OLED, 128GB, Càmera 12MP", "preu": 699.99},
   "Model Y": {"especificacions": "Pantalla LCD, 64GB, Càmera 8MP", "preu": 499.99},
   "Model Z": {"especificacions": "Pantalla AMOLED, 256GB, Càmera 16MP", "preu": 899.99}
}

#Convertimos el diccionario en una lista de documentos
documents = [
    Document(page_content=f"{model}: {info['especificacions']}, Precio: {info['preu']}")
    for model, info in productos.items()
]

#Preprocesamiento del código
from langchain.text_splitter import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                               chunk_size=2000,
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)

In [17]:
#Se define la clave personal de OpenaAI
api_key="Aquí va la APIkey de OpenAI"

#Creación de las incrustaciones
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=(),openai_api_key=api_key))

# Parámetros de similaridad
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8},
)

In [18]:
#Se instancia el modelo gpt 3.5-turbo
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=api_key)
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)
qa=ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [19]:
#Una pregunta de un cliente y las indicaciones de como ha de ser la respuesta
pregunta="Quines són les especificacions del model Y i el seu preu?"
indicaciones=" La construcción de la respuesta ha de ser similar a : 'Els descriptius del Model X inclouen una pantalla OLED de gran qualitat, 128GB d'emmagatzematge intern i una càmera de 12MP. El preu d'aquest producte és de 699.99$. Si tens més preguntes o necessites més informació, estic aquí per ajudar-te.'"
question=pregunta+indicaciones
result= qa(question)
result['answer']


"Els descriptius del Model Y inclouen una pantalla LCD, 64GB d'emmagatzematge intern i una càmera de 8MP. El preu d'aquest producte és de 499.99$. Si tens més preguntes o necessites més informació, estic aquí per ajudar-te."